In [ ]:
import numpy as np

In [ ]:
#a=np.loadtxt("BMI_4PSK")


level=0
filename="/home/kaneko/Dropbox/programming/geometric_shaping/polar_1024_512_4PSK_RCA_SC_level{}".format(level)
#/home/kaneko/Dropbox/programming/geometric_shaping/result/Polar/polar_1024_512_4QAM_RCA_SC
b=np.loadtxt(filename)

print(a[:,level+1])

b[:,0]=a[:8,level+1]/2

print(b)

In [ ]:
np.savetxt(filename,b)

In [ ]:
np.arange(0,30,0.5)

In [ ]:
EsNodB_list=np.arange(0,30,0.5)
for EsNodB in EsNodB_list:
    filename="BMI/256PSK_{}".format(EsNodB)
    a=np.loadtxt(filename)
    a/=10
    np.savetxt(filename,a)


In [ ]:
M=4
EsNodB_list=np.arange(0,30,0.5)
print(len(EsNodB_list))
res=np.zeros((2,60))
print(res.shape)
for i,EsNodB in enumerate(EsNodB_list): 
    filename="BMI/{}PSK_{}".format(M,EsNodB)
    a=np.loadtxt(filename)
   #print(i)
    res[:,i]=a
    
print(res)



In [ ]:
filename="BMI_16PSK"
with open(filename,'w') as f:

    for i in range(res.shape[1]):
        print(str(EsNodB_list[i]),str(res[0,i]),str(res[1,i]),str(res[2,i]),str(res[3,i]),file=f)

In [ ]:
a=np.loadtxt("BMI_4QAM")
a[:,1]/2
res=np.stack([a[:,0],a[:,1]/2,a[:,1]/2],axis=1)
np.savetxt("BMI_4QAM",res)

In [ ]:
a=np.loadtxt("16QAM")


level=0
filename="/home/kaneko/Dropbox/programming/geometric_shaping/polar_1024_512_16QAM_RCA_SC_level{}".format(level)
#/home/kaneko/Dropbox/programming/geometric_shaping/result/Polar/polar_1024_512_4QAM_RCA_SC
b=np.loadtxt(filename)

print(a[:,level+1])
print(b)

b[:,0]=a[:17,level+1]

print(b)



In [ ]:
np.savetxt(filename,b)

In [ ]:
from calc_capacity import make_AMI
from calc_capacity import make_BMI

In [ ]:
make_BMI(1.0,16)

In [ ]:
ver=1 #1:AMI 2:BMI

ERR_accept=10**-3

err=100
EsNodB=3
M=256
R=1/2
while err>ERR_accept:
    if ver==1:
        tmp=make_AMI(EsNodB,M)
    elif ver==2:
        tmp=make_BMI(EsNodB,M)
    err=abs(np.log2(M)*R-tmp)
    EsNodB+=(np.log2(M)*R-tmp)*0.1
    print(EsNodB,err,tmp)
    
print(EsNodB)

In [12]:
import math
import numpy as np
import cupy as cp
#pool = cp.cuda.MemoryPool(cp.cuda.malloc_managed)
#cp.cuda.set_allocator(pool.malloc)
import sys
import os
from scipy.stats import norm
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.path.dirname('__file__'), '..'))

from modulation.modulation import QAMModem
from modulation.modulation import PSKModem


# In[101]:


def add_AWGN_GPU(constellation,No):
  # AWGN雑音の生成
  noise = cp.random.normal(0, math.sqrt(No / 2), (len(constellation)))           + 1j * cp.random.normal(0, math.sqrt(No / 2), (len(constellation)))

  # AWGN通信路 = 送信シンボル間干渉が生じないような通信路で送信
  RX_constellation = constellation + noise

  # 以下のprint関数の出力を表示すると、Noとほぼ一致するはず
  #print(cp.dot(noise[0, :], cp.conj(noise[0, :]))/bit_num)

  return RX_constellation


# In[102]:


def make_AMI(EsNodB,M):
    modem_ver=0 #0:QAMModem 1:PSKModem
    
  
    EsNo = 10 ** (EsNodB / 10)
    No=1/EsNo
    count_num=10000000

    #make info matrices
    info=cp.random.randint(0,M,count_num)

    #make constellation
    if M!=2:
        if modem_ver==0:
            modem=QAMModem(M)
        elif modem_ver==1:
            modem=PSKModem(M)
        else:
            print("modem version error")
    
    
        tmp=modem.code_book
        symbol=cp.zeros(M,dtype=complex)
        for i in tmp:
            symbol[modem.bin2de(i)]=tmp[i]
        
    elif M==2:
        symbol=cp.array([-1,1])
        
    mat_symbol=cp.tile(symbol,(count_num,1))
    const=cp.take_along_axis(mat_symbol,info[:,None],axis=1)[:,0]
    

    #if cp.any(symbol==const)!=True:
        #print("error")
        #print(symbol)
        #print(const)

    RX_const=add_AWGN_GPU(const,No)
        
    num=cp.sum(cp.exp(-1*cp.abs(np.tile(RX_const,(len(symbol),1))-symbol.reshape(-1,1))**2/No),axis=0)
    
    den=cp.exp(-1*cp.abs(RX_const-const)**2/No)
    H=cp.sum(cp.log2(num/den))
    H/=count_num
    res=math.log2(M)-H
    return res

In [13]:
make_AMI(0,16)

array(0.98973057)

In [22]:
import numpy as np

M=16
#EsNodB=10.0
EsNodB_list=np.arange(0,21,0.5)
res=np.zeros(len(EsNodB_list))
print(EsNodB_list)
for i in range(len(EsNodB_list)):
    EsNodB=EsNodB_list[i]
    print(EsNodB)
    filename="BMI/{}QAM_{}".format(M,EsNodB)
    a=np.loadtxt(filename)
    #print(a)
    #res=make_AMI(EsNodB,M)
    #print(res[i])
    #print(res)
    #res_seq=a
    ##from IPython.core.debugger import Pdb; Pdb().set_trace()
    a[1]*=1.1
    a[3]*=1.1    
    #if np.any(a>res_seq):
    #    print("error!!")
    #print(a)
    filename_res="BMI_polar/{}QAM_{}".format(M,EsNodB)
    np.savetxt(filename_res,a)
np.savetxt("ami",res)

[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5 16.  16.5 17.  17.5 18.  18.5 19.  19.5 20.  20.5]
0.0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5
5.0
5.5
6.0
6.5
7.0
7.5
8.0
8.5
9.0
9.5
10.0
10.5
11.0
11.5
12.0
12.5
13.0
13.5
14.0
14.5
15.0
15.5
16.0
16.5
17.0
17.5
18.0
18.5
19.0
19.5
20.0
20.5
